In [50]:
install.packages("mclogit")

also installing the dependency ‘memisc’

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [51]:
library(mclogit)

Loading required package: Matrix


In [62]:
mydata = read.csv("buffalo.csv")
print(mydata)

  grass water trees label frame
1     2     5     8     1     1
2    10     2     1     1     2
3     2     4     1     0     1
4     3     2     2     0     2
5    10     7     1     0     1
6     9     5     4     0     2
7     6     5     3     0     1
8     4     4     3     0     2


In [155]:
test <- mclogit(cbind(mydata$label,mydata$frame) ~ grass + water + trees,data=mydata)
coef(test)


Iteration 1 - Deviance = 0.9991882
Iteration 2 - Deviance = 0.3608208
Iteration 3 - Deviance = 0.135204
Iteration 4 - Deviance = 0.05049028
Iteration 5 - Deviance = 0.01873359
Iteration 6 - Deviance = 0.006921847
Iteration 7 - Deviance = 0.002551872
Iteration 8 - Deviance = 0.0009397564
Iteration 9 - Deviance = 0.00034589
Iteration 10 - Deviance = 0.0001272764
Iteration 11 - Deviance = 4.682777e-05
Iteration 12 - Deviance = 1.722793e-05
Iteration 13 - Deviance = 6.33797e-06
Iteration 14 - Deviance = 2.331639e-06
Iteration 15 - Deviance = 8.577672e-07
Iteration 16 - Deviance = 3.155559e-07
Iteration 17 - Deviance = 1.160867e-07
Iteration 18 - Deviance = 4.270594e-08
Iteration 19 - Deviance = 1.571064e-08
Iteration 20 - Deviance = 5.779623e-09
Iteration 21 - Deviance = 2.126204e-09
Iteration 22 - Deviance = 7.821872e-10
Iteration 23 - Deviance = 2.877505e-10
converged


grass      water      trees 
  4.631775 -14.880608   8.363130

### Plug betas back into probability function
    - should get 1 or value close to 1 for chosen steps (y23 and y31)
    - should get 0 for all alternative steps

In [150]:
betas <- c(4.63, -14.88, 8.36)
betas_paper <- c(1.9, -6.2, 3.6)

y23 <- c(2, 5, 8)
y21 <- c(2, 4, 1)
y22 <- c(10, 7, 1)
y24 <- c(6, 5, 3)

y31 <- c(10, 2, 1)
y32 <- c(3, 2, 2)
y33 <- c(9, 5, 4)
y34 <- c(4, 4, 3)

step1 <- exp(betas%*%y23)/sum(exp(betas%*%y21), exp(betas%*%y22), exp(betas%*%y23), exp(betas%*%y24))
step1_paper <- exp(betas_paper%*%y23)/sum(exp(betas_paper%*%y21), exp(betas_paper%*%y22), exp(betas_paper%*%y23), exp(betas_paper%*%y24))

print(step1)
print(step1_paper)


     [,1]
[1,]    1
          [,1]
[1,] 0.9999696


In [151]:
step1_alt1 <- exp(betas%*%y21)/sum(exp(betas%*%y23), exp(betas%*%y21), exp(betas%*%y22), exp(betas%*%y24))
step1_paper_alt1 <- exp(betas_paper%*%y21)/sum(exp(betas_paper%*%y21), exp(betas_paper%*%y23), exp(betas_paper%*%y22), exp(betas_paper%*%y24))
print(step1_alt1)
print(step1_paper_alt1)

step1_alt2 <- exp(betas%*%y22)/sum(exp(betas%*%y23), exp(betas%*%y22), exp(betas%*%y21), exp(betas%*%y24))
step1_paper_alt2 <- exp(betas_paper%*%y22)/sum(exp(betas_paper%*%y23), exp(betas_paper%*%y22), exp(betas_paper%*%y21), exp(betas_paper%*%y24))
print(step1_alt2)
print(step1_paper_alt2)

step1_alt3 <- exp(betas%*%y24)/sum(exp(betas%*%y23), exp(betas%*%y22), exp(betas%*%y24), exp(betas%*%y21))
step1_paper_alt3 <- exp(betas_paper%*%y24)/sum(exp(betas_paper%*%y23), exp(betas_paper%*%y22), exp(betas_paper%*%y24), exp(betas_paper%*%y21))
print(step1_alt3)
print(step1_paper_alt3)

             [,1]
[1,] 1.115293e-19
             [,1]
[1,] 5.602626e-09
             [,1]
[1,] 5.581498e-23
             [,1]
[1,] 1.869779e-10
             [,1]
[1,] 7.755762e-11
             [,1]
[1,] 3.043156e-05


### Calculate log-likelihood difference

In [137]:
B <- as.vector(c(4.63, -14.88, 8.36))
Bnull <- as.vector(c(0,0,0))
used <- matrix(c(2,10,5,2,8,1), nrow=2, ncol=3)
alt1 <- matrix(c(2,3,4,2,1,2), nrow=2, ncol=3)
alt2 <- matrix(c(10,9,7,5,1,4), nrow=2, ncol=3)
alt3 <- matrix(c(6,4,5,4,3,3), nrow=2, ncol=3)

In [148]:
nLL <- function(B) {
    func <- function(x) {
    return (-log(exp(B%*%used[x,]) + exp(B%*%alt1[x,]) + exp(B%*%alt2[x,]) + exp(B%*%alt3[x,])))
    }

LL <- sum(apply(used, 1, function(x)B%*%x)) + sum(sapply(seq(length =nrow(used)), func))
    return(-LL)
}

neglogliks <- vector()
neglogliks <- c(neglogliks, nLL(Bnull) - nLL(B))

In [149]:
print(neglogliks)

[1] 2.772589
